In [ ]:
import constants as c
c.MU0

In [ ]:
# courant criteria
import numpy as np
1/np.sqrt(3*(1/6e-3)**2)/299792458

In [ ]:
import numpy as np
import constants as c
c.MU0, c.EP0, 1/np.sqrt(c.MU0*c.EP0)

In [ ]:
import structuredpoints_curtesian_cupy as fdtd
import structuredpoints_curtesian_ui as ui
from base import sin3

freq = 1e+9 # [GHz]

class MySolver(fdtd.Solver):
    def sequence(self):
        ex, ey, ez = self.yee.e
        hx, hy, hz = self.yee.h
        i, j, k = 50, 50, 25
        self.time.progress_half_step()
        self.calc_h()
        print('hx', hx[i,j-1:j+1,k])
        print('hy', hy[i-1:i+1,j,k])
        self.time.progress_half_step()
        self.calc_e()
        ez[50, 50, 26:37] = 0.
        ez[50, 50, 14:25] = 0.
        self.source_z(i, j, k, sin3(self.time, freq), r=50.)
        print('ez', ez[i-1:i+2,j-1:j+2,k])


s = MySolver()
s.grid.dimensions = 100, 100, 51 # [cells]
s.grid.spacing = 6e-3, 6e-3, 6e-3 # [mm]
s.time.dt = .01e-9 # [s]
s.prepare()

for i in range(2):
    s.calculate()

In [ ]:
import structuredpoints_curtesian_cupy as fdtd
import structuredpoints_curtesian_ui as ui
from base import sin3
import time

#f = fdtd.Fields()
#f.alloc(1000,1000,100)

freq = 1e+9 # [GHz]

class MySolver(fdtd.Solver):
    def sequence(self):
        ex, ey, ez = self.yee.e
        self.time.progress_half_step()
        self.calc_h()
        self.time.progress_half_step()
        self.calc_e()
        ez[50, 50, 26:37] = 0.
        ez[50, 50, 14:25] = 0.
        self.source_z(50, 50, 25, sin3(self.time, freq), r=50.)

s = MySolver()

s.grid.dimensions = 100, 100, 51 # [cells]
s.grid.spacing = 6e-3, 6e-3, 6e-3 # [mm]
s.time.dt = .01e-9 # [ns]

s.prepare()

u = ui.UI(s)
u.mapper = ui.LinearColorMapper(palette='Viridis256', low=-1, high=1)
u.slices = (50, slice(None), slice(None))
u.render()